In [19]:
import numpy as np
embeddings_index = {}
with open('numberbatch-en-17.06.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 417195


In [20]:
import pickle as pk
import random
from tqdm import tqdm
import sys
import unicodedata
import time
import os
import nltk
import tensorflow as tf
import numpy as np

punctuation = dict.fromkeys([i for i in range(sys.maxunicode)
                                 if unicodedata.category(chr(i)).startswith('P')])

with open('data_pita.pkl','rb') as f:
    data_set = pk.load(f)
    
    


with open('vocab_pita.pkl','rb') as f:
    vocab_ = pk.load(f)


In [ ]:
# data_set =[]
# vocab_ =  []

# for i,j in tqdm(open_data.items()):
#     kl=[i.lower() for i in nltk.word_tokenize(i.translate(punctuation))]
#     data_set.append((kl,j))
#     vocab_.extend(kl)

    
# # with open('vocab_soha.pkl','wb') as f:
# #     pk.dump(list(set(vocab_)),f)
    

    
    
# # with open('data_soha.pkl','wb') as f:
# #     pk.dump(data_set,f)

In [21]:
vocab_o = ['PAD'] + list(set(vocab_))
vocab_to_int = {j:i for i,j in enumerate(vocab_o)}
int_to_vocab ={k:m for m,k in vocab_to_int.items()}



In [22]:
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

2246


In [24]:
import random
encoded_data =[]
for i in tqdm(data_set):
        encoded_data.append(([vocab_to_int[m] for m in i[0] if m in vocab_to_int],i[1]))
        
random.shuffle(encoded_data)

test_data = encoded_data[:100]
train_data = encoded_data[100:]

np.save('testt',test_data)


targ =[]
for i in encoded_data:
    targ.append(i[1])
    
print(targ.count(1))
print(targ.count(0))



100%|██████████| 7086/7086 [00:00<00:00, 222669.70it/s]

3995
3091


In [25]:
categories_f =[0, 1,]

real_ca = {0:'negative',1:'postive'}


def get_batch(i,batch_size,test=None):
    if test==0:
        
        encoded_data_r = test_data[i*batch_size:(i+1)*batch_size]
    else:
        encoded_data_r = encoded_data[i*batch_size:(i+1)*batch_size]

        

    sentence=[]
    sentiment =[]
    for i in encoded_data_r:
        sentence.append(i[0])
        sentiment.append(categories_f.index(i[1]))
        
    max_len = max([len(i) for i in  sentence])
    
    pad_data = [m + [0]*(max_len -len(m)) if len(m)<max_len else m for m in sentence]
    
    return np.array(pad_data) , np.array(sentiment)



In [33]:
__version__ = "1.0"
__maintainer__ = "Monk"



tf.reset_default_graph()

class lstm_Attention_with_Pos(object):
    
    
    def __init__(self,vocab_size,num_cell,dropout_value,category_size):
        
        
        
        #placeholders
        sentence = tf.placeholder(name='input_data',shape=[None,None],dtype=tf.int32)
#         pos_     = tf.placeholder(name='pos',shape=[None,None],dtype=tf.int32)
        intents  = tf.placeholder(name='labels',shape=[None],dtype=tf.int32)
        mode =  tf.placeholder(name='dropout',shape=(),dtype=tf.int32)
        
        
        
        
        
        self.placeholder = {
            
                            'input_d':sentence,
                            'labels':intents,
#                              'pos':pos_,
                            'dropout':mode
                            }
        

        
        
#         embedding_matrix = tf.get_variable(name='word_embedding',
#                                            shape=[vocab_size,embedding_dim],
#                                            dtype=tf.float32,
#                                            initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        final_input = tf.nn.embedding_lookup(word_embedding_matrix,sentence)
        
        dropout = tf.cond(
                          tf.equal(mode,0),
                          lambda : dropout_value, 
                          lambda : 0.
                         )
        
#         pos_embedding = tf.get_variable(name='pos_embedding',
#                                            shape=[pos_vocab_size,pdim],
#                                            dtype=tf.float32,
#                                            initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        
#         pos_lookup = tf.nn.embedding_lookup(pos_embedding, pos_ )
        
#         final_input = tf.concat([embedd_word2vec,pos_lookup],axis=-1)
        
        
                
        
        
        
        
        sequence_length = tf.count_nonzero(sentence,axis=-1)
        
        
        with tf.variable_scope('forward_cell'):
                lstm_cell_fw = tf.contrib.rnn.LSTMCell(num_cell,forget_bias=1.0)
                dropout_wrapper_fw = tf.contrib.rnn.DropoutWrapper(lstm_cell_fw,output_keep_prob=1. - dropout_value)
        
        
        with tf.variable_scope('backward_cell'):
            lstm_cell_bw = tf.contrib.rnn.LSTMCell(num_cell,forget_bias=1.0)
            dropout_wrapper_bw = tf.contrib.rnn.DropoutWrapper(lstm_cell_bw,output_keep_prob=1.- dropout_value)
        
        
        with tf.variable_scope('bi_lstm') as scope:
            output , last_state     = tf.nn.bidirectional_dynamic_rnn(dropout_wrapper_fw,
                                                      dropout_wrapper_bw,
                                                      final_input,
                                                      sequence_length=sequence_length,
                                                      dtype=tf.float32)
        
        logits = tf.concat(output,2)
        #ex . 12 x 10 x 24
        
        
        
        
        #Attention_layer
        
        
        #ex : 12x10x24 . ===> 120 x 24
        input_reshape = tf.reshape(logits,[-1,num_cell*2])
        
        
        #num_cell = 12
        #ex : # 24 x 1
        attention_size = tf.get_variable(name='attention_size',
                                         shape=[2*num_cell,1],
                                         dtype=tf.float32,
                                         initializer=tf.random_uniform_initializer(-0.01,0.01))
        # bias 1
        bias          = tf.get_variable(name='bias',shape=[1],
                                        dtype=tf.float32,
                                        initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        
        #projection without activation 
        #ex : 120x24 matmul 24x 1 ==> 120x1
        attention_projection = tf.add(tf.matmul(input_reshape,attention_size),bias)
        
        
        #reshape . 120x1 ==> 12x10x1 (shape of input )
        output_reshape = tf.reshape(attention_projection,[tf.shape(sentence)[0],tf.shape(sentence)[1],-1])
        
        #softmax over logits 12x10x1
        attention_output = tf.nn.softmax(output_reshape,dim=1)
        
        
        #reshape as input 12x10
        attention_visualize = tf.reshape(attention_output,
                                         [tf.shape(sentence)[0],
                                          tf.shape(sentence)[1]],
                                         name='Plot')
        
        
        # 12x10x1 multiply 12x10x24  == > 12x10x24
        attention_projection_output = tf.multiply(attention_output,logits)
        
        #reduce across time 120x10x24 ==> 12x24
        Final_output = tf.reduce_sum(attention_projection_output,1)
        
        
        
        #fully_connected_layer
        
        f_c = tf.get_variable(name='f_c_layer',
                              shape=[2*num_cell,category_size],
                              dtype=tf.float32,
                              initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        
        f_bias = tf.get_variable(name='fc_bias',
                                 shape=[category_size],
                                dtype=tf.float32,
                                 initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        output_projection = tf.add(tf.matmul(Final_output,f_c),f_bias)
        
        
        
        probability_dist  = tf.nn.softmax(output_projection,name='pred')
        
        prediction  = tf.argmax(probability_dist,axis=-1)
        
        
        #as_usual_stuff_
        cross_entropy  = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=intents,logits=output_projection)
        loss = tf.reduce_mean(cross_entropy)
        accuracy_calculation = tf.reduce_mean(tf.cast(tf.equal(tf.cast(prediction,tf.int32),intents),tf.float32))
        
        
        
        self.output = {  'loss':loss,
                         'accuracy':accuracy_calculation,
                         'prediction':prediction,
                         'probai':probability_dist,
                         'logits':output_projection,
                         'attention_visualize':attention_visualize
            
                       }
        
        
        # this guy is hero of this movie
        self.train = tf.train.AdamOptimizer().minimize(loss)
        

# def execute_model(model):
    
#     with tf.Session() as sess:
        
#         sess.run(tf.global_variables_initializer())
        
#         for i in range(100):
#             outp,train = sess.run([model.output,model.train],feed_dict={model.placeholders['input_d']:np.random.randint(0,3,[12,10]),
#                                                 model.placeholders['labels']:np.random.randint(0,2,[12,]),
                                                
# #                                                 model.placeholders['pos']: np.random.randint(0,3,[12,10]),
#                                                 model.placeholders['dropout']:0,
#                                                                        })
#             print(outp['loss'],outp['accuracy'],outp['prediction'].shape,outp['attention_visualize'].shape)
        
        
# if __name__ == '__main__':
    
#     model = lstm_Attention_with_Pos(130,100,12,0.5,120,22,12)
#     execute_model(model)


In [32]:
epoch = 1
def evaluate_(model, batch_size=10):

    sess = tf.get_default_session()

    # batch_data = test_data_in
    # batch_labesl = train_labels

    iteration = len(test_data) // batch_size

    accuracy = []

    for i in range(iteration):
        input_test,labels = get_batch(i,10,test=0)

        network_out = sess.run(model.output, feed_dict={model.placeholder['input_d']: input_test,
                                                        model.placeholder['labels']: labels,
                                                        model.placeholder['dropout']: 1})

        accuracy.append(network_out['accuracy'])
    return np.mean(np.array(accuracy))


def train_model(model, batch_size=100):

    saver = tf.train.Saver()
    with tf.Session() as sess:

        sess.run(tf.global_variables_initializer())

        iteration = len(encoded_data) // batch_size
        print("iteration", iteration)
        

        for i in range(epoch):

            for j in range(iteration):
                
                input_train,labels = get_batch(i,100)

                

                network_out, _ = sess.run([model.output, model.train],
                                          feed_dict={model.placeholder['input_d']: input_train,
                                                     model.placeholder['labels']: labels,
                                                     model.placeholder['dropout']: 0})

                print({'epoch': i,
                       'iteration': j,
                       'training_loss': network_out['loss'],
                       'training_accuracy': network_out['accuracy']
                       })
                

            print({'epoch': i, 'test_accuracy': evaluate_(model)})
            with open('epochandresult', 'a') as f:
                f.write(str({'epoch': i, 'test_accuracy': evaluate_(model)}) + '\n')
            os.system('mkdir ' + str(i) + 'epoch')
            saver.save(sess, '/Users/monk/Desktop/Word_cloud_model/movies/punjab_grill/haha/lk' + str(i) + 'epoch' + '/' + str(i))
            time.sleep(5)


if __name__ == "__main__":
    model = lstm_Attention_with_Pos(len(vocab_),200,0.5,2)

    train_model(model)

iteration 70
{'epoch': 0, 'iteration': 0, 'training_loss': 0.69257826, 'training_accuracy': 0.6}
{'epoch': 0, 'iteration': 1, 'training_loss': 0.69146955, 'training_accuracy': 0.6}
{'epoch': 0, 'iteration': 2, 'training_loss': 0.69036394, 'training_accuracy': 0.6}
{'epoch': 0, 'iteration': 3, 'training_loss': 0.6890548, 'training_accuracy': 0.6}
{'epoch': 0, 'iteration': 4, 'training_loss': 0.6873149, 'training_accuracy': 0.62}
{'epoch': 0, 'iteration': 5, 'training_loss': 0.68481845, 'training_accuracy': 0.62}
{'epoch': 0, 'iteration': 6, 'training_loss': 0.68165475, 'training_accuracy': 0.62}
{'epoch': 0, 'iteration': 7, 'training_loss': 0.6776003, 'training_accuracy': 0.62}
{'epoch': 0, 'iteration': 8, 'training_loss': 0.67339295, 'training_accuracy': 0.62}
{'epoch': 0, 'iteration': 9, 'training_loss': 0.6692241, 'training_accuracy': 0.63}
{'epoch': 0, 'iteration': 10, 'training_loss': 0.66206115, 'training_accuracy': 0.63}
{'epoch': 0, 'iteration': 11, 'training_loss': 0.65178114, 

In [ ]:
import tensorflow as tf
import pickle  as pk
import nltk
import sys
import unicodedata

with open('vocab_pita.pkl','rb') as f:
    vocab_o = pk.load(f)

vocab_to_int = {j:i for i,j in enumerate(vocab_o)}
int_to_vocab ={k:m for m,k in vocab_to_int.items()}

punctuation=dict.fromkeys([i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P')])	
	
def remove_pun(text):	
    #removing punctuation	
    return [i.lower() for i in nltk.word_tokenize(text.translate(punctuation))]	


In [36]:
# sess =  tf.Session()	
# saver = tf.train.import_meta_graph('16epoch/16.meta')	
# restore = saver.restore(sess,tf.train.latest_checkpoint('16epoch/'))	
	
# graph=tf.get_default_graph()	
	
# query= graph.get_tensor_by_name("input_data:0")	
# result=graph.get_tensor_by_name("pred:0")	
# plot=graph.get_tensor_by_name("Plot:0")	





# def encode_query(text_):	
#     query=remove_pun(text_)	
#     sequence_data=[]	
#     for i in query:	
	
#         if i in vocab_to_int:	
#             sequence_data.append(vocab_to_int[i])	
	
#     return sequence_data


# labels_dict = {0:'neutal',1:'postive',2:'negative'}
# categories_f =[0, 1,-1]


    	
# def predict(user_query_,sess=sess):	
# #         user_query_p = user_query_.split()
#         user_query = user_query_
#         feed_dict = {query:[user_query]}	
        	
#         prediction,plot_a = sess.run([result,plot],feed_dict=feed_dict)	
        
# #         print(prediction)
#         return labels_dict[prediction[0].tolist().index(max(prediction[0].tolist()))]
# #         print(list(zip(user_query_p,plot_a[0])))
# #         pred=labels_dict[prediction[0].index(max(prediction[0]))]	
# #         print(plot_a,user_query_)
# #         return { 	
# #                  'Adverse_Event_Flag': pred ,	
# #                  'Probability'       : max(prediction[0])	
# #                }	



In [ ]:
wrong=0
for i in test_data:
    if predict(i[0])!=labels_dict[categories_f.index(i[1])]:
        wrong+=1
print(wrong)
    